In [1]:
import pandas as pd,numpy as np
from sklearn.metrics import auc
pd.set_option('max_columns',30) 

In [2]:
def process_data():
    df_op11=pd.read_csv('F:\\study\\ml\\DataSet\\TianChengJinRong\\round1-new\\operation_train_new.csv')
    df_tr11=pd.read_csv('F:\\study\\ml\\DataSet\\TianChengJinRong\\round1-new\\transaction_train_new.csv')
    #df_tag1=pd.read_csv('F:\\study\\ml\\DataSet\\TianChengJinRong\\round1-new\\tag_train_new.csv')
    df_op12=pd.read_csv('F:\\study\\ml\\DataSet\\TianChengJinRong\\round1-new\\operation_round1_new.csv')
    df_tr12=pd.read_csv('F:\\study\\ml\\DataSet\\TianChengJinRong\\round1-new\\transaction_round1_new.csv')
    #df_tag1=pd.read_csv('F:\study\\ml\\DataSet\\TianChengJinRong\\tag_TRAIN.csv')
        
    df_op1=pd.concat([df_op11,df_op12],axis=0)
    df_tr1=pd.concat([df_tr11,df_tr12],axis=0)
    
    df_op1=df_op1.drop_duplicates()
    df_tr1=df_tr1.drop_duplicates()
    #df_tag1=df_tag1.drop_duplicates()

    df_op=df_op1.groupby('UID').transform(lambda x:x.fillna( x.mode()[0] if x.mode().shape[0] > 0 else 'None'))

    df_tr=df_tr1.groupby('UID').transform(lambda x:x.fillna( x.mode()[0] if x.mode().shape[0] > 0 else 'None'))

    df_op=pd.concat([df_op1['UID'],df_op],axis=1)



    df_tr=pd.concat([df_tr1['UID'],df_tr],axis=1)

    df_op[df_op['UID']==10006]['device_code1']

    df_op1[df_op1['UID']==10006]['device_code1']

    a=df_op[['UID','day']].groupby('UID').sum().index
    b=df_tr[['UID','day']].groupby('UID').sum().index

    data_train=pd.DataFrame()
    data_train['UID']=np.array(a | b)

    data_train['op_count']=0
    data_train.loc[data_train['UID'].isin(df_op[['UID','day']].groupby('UID').sum().index),['op_count']]=\
    df_op[['UID','day']].groupby('UID').count().values

    data_train['tr_count']=0
    data_train.loc[data_train['UID'].isin(df_tr[['UID','day']].groupby('UID').sum().index),['tr_count']]=\
    df_tr[['UID','day']].groupby('UID').count().values

    op_days=pd.DataFrame()
    op_days['UID']=df_op['UID']
    op_days=pd.concat([op_days,pd.get_dummies(df_op['day'],prefix='op_day')],axis=1)
    op_days_sum=op_days.groupby(by='UID').sum()

    tr_days=pd.DataFrame()
    tr_days['UID']=df_tr['UID']
    tr_days=pd.concat([tr_days,pd.get_dummies(df_tr['day'],prefix='tr_day')],axis=1)
    tr_days_sum=tr_days.groupby(by='UID').sum()

    data_train1=data_train

    data_train1=pd.merge(data_train1,op_days_sum,how='left',on='UID')


    from sklearn.preprocessing import LabelEncoder
    le=LabelEncoder()

    def op_object(train):
        for i in ['mode','os','version','']:
            train[i]=train[i].factorize()[0]
    #         test[i]=test[i].factorize()[0]
            train[i]=le.fit_transform(train[i])
    #         test[i]=le.fit_transform(test[i]) 
        return train


    op_col2=[i for i in df_op.columns.values ]
    op_col2.remove('time')
    op_col2.remove('day')
    op_col2.remove('UID')

    def fun_ob2le(train,names):
        for i in names:
            train[i]=pd.factorize(train[i])[0]
            train[i]=le.fit_transform(train[i])
        return train

    df_op1=df_op.copy()
    df_op1=fun_ob2le(df_op1,op_col2)
#     df_op1.head()

#     df_op1[df_op1['UID']==10006]['device_code1']

    df_op2=pd.DataFrame()
    df_op2=df_op['UID']
    for i in ['success','os','version','mode']:
        df_op2=pd.concat([df_op2,pd.get_dummies(df_op1[i],prefix=i)],axis=1)   
    df_op2.head()

    df_op3=df_op2.groupby('UID',as_index=False).sum()


    #del df_ios

    df_ios=df_op1[['UID','device_code1','device_code3']].groupby('UID',as_index=False).agg('max')

    df_ios['device_code1']=df_ios['device_code1'].apply(lambda x: 1 if x>0 else 0)
    df_ios['device_code3']=df_ios['device_code3'].apply(lambda x: 1 if x>0 else 0)


    df_ios.rename(columns={'device_code1':'is_android','device_code3':'is_ios'},inplace=True)


    data_train2=pd.merge(data_train1,df_ios,how='left',on='UID')

    op1_col=df_op.columns.values.tolist()
    op1_col1=[op1_col[0]]
    op1_col1.extend(op1_col[7:9])
    op1_col1.extend(op1_col[12:])


    df_op4=df_op1[op1_col1].groupby(by=['UID']).agg('nunique').iloc[:,1:]


    #########  data_train2,merge df_op3,df_op4,

    data_train2.merge(df_op3,how='left',on='UID')

    data_train2.merge(df_op4,how='left',on='UID')

    data_train3=data_train2.merge(df_op3,how='left',on='UID',)
    data_train3=data_train3.merge(df_op4,how='left',on='UID')



    df_tr1=df_tr
    # df_tr1_col=df_tr.columns.values.tolist()
    for i in df_tr.columns.values.tolist():
        if 'UID' not in i:
            df_tr1.rename(columns={i:'tr_'+i},inplace=True)



    ob2le_name=[]
    for i in df_tr1.columns.values.tolist():
        if  i not in ['UID','tr_day','tr_time','tr_trans_amt','tr_bal']:
            ob2le_name.append(i)
    ob2le_name        

    df_tr2=pd.DataFrame()
    df_tr2['UID']=df_tr1['UID']
    df_tr2=pd.concat([df_tr2,fun_ob2le(df_tr1[ob2le_name],ob2le_name)],axis=1)



    # df_tr3=
    df_tr3=df_tr2.groupby('UID').agg('nunique').iloc[:,1:]

    data_train4=data_train3.merge(df_tr3,how='left',on='UID')




    df_tr3_price=df_tr1[['UID','tr_trans_amt','tr_bal']].groupby('UID').agg(['min','max','mean','median'])

    data_train5=data_train4.merge(df_tr3_price,how='left',on='UID')



    for i in [('tr_trans_amt', 'min'), ('tr_trans_amt', 'max'),('tr_trans_amt', 'mean'),('tr_trans_amt', 'median'),('tr_bal', 'min'), ('tr_bal', 'max'), ('tr_bal', 'mean'),('tr_bal', 'median')]:
        data_train5.rename(columns={i:i[0]+'_'+i[1]},inplace=True)



    def fun_last_se(arr):
        try:
            arr=arr.sort_values()
            return (arr.iloc[-1] - arr.iloc[-2]).seconds
        except:
            return np.nan


    import datetime
    import time


    # pd.to_datetime(df_tr[['tr_day','tr_time']].apply(lambda x,y:'2017-10-'+str(x)+' '+str(y)))

    df_tr['tr_day1']='2017-10-'+df_tr['tr_day'].apply(lambda x:str(x) if x>=10 else '0'+str(x))


    df_tr['tr_time1']=df_tr['tr_day1']+' '+df_tr['tr_time']


    df_tr['tr_time2']=df_tr['tr_time1'].apply(lambda x:pd.to_datetime(x,format='%Y-%m-%d %H:%M:%S'))

    def fun_last2_se(arr):
        try:
            arr=arr.sort_values()
            return (arr.iloc[-2] - arr.iloc[-3]).seconds
        except:
            return np.nan

    def fun_last3_se(arr):
        try:
            arr=arr.sort_values()
            return (arr.iloc[-3] - arr.iloc[-4]).seconds
        except:
            return np.nan
    def fun_last4_se(arr):
        try:
            arr=arr.sort_values()
            return (arr.iloc[-4] - arr.iloc[-5]).seconds
        except:
            return np.nan

    def fun_last5_se(arr):
        try:
            arr=arr.sort_values()
            return (arr.iloc[-5] - arr.iloc[-6]).seconds
        except:
            return np.nan

    def fun_last6_se(arr):
        try:
            arr=arr.sort_values()
            return (arr.iloc[-6] - arr.iloc[-7]).seconds
        except:
            return np.nan 

    def fun_last7_se(arr):
        try:
            arr=arr.sort_values()
            return (arr.iloc[-7] - arr.iloc[-8]).seconds
        except:
            return np.nan   

    #############################################

    def fun_last_day(arr):
        try:
            arr=arr.sort_values()
            return (arr.iloc[-1] - arr.iloc[-2]).days
        except:
            return np.nan
    def fun_last2_day(arr):
        try:
            arr=arr.sort_values()
            return (arr.iloc[-2] - arr.iloc[-3]).days
        except:
            return np.nan

    def fun_last3_day(arr):
        try:
            arr=arr.sort_values()
            return (arr.iloc[-3] - arr.iloc[-4]).days
        except:
            return np.nan
        
    def fun_last4_day(arr):
        try:
            arr=arr.sort_values()
            return (arr.iloc[-4] - arr.iloc[-5]).days
        except:
            return np.nan

    def fun_last5_day(arr):
        try:
            arr=arr.sort_values()
            return (arr.iloc[-4] - arr.iloc[-5]).days
        except:
            return np.nan

    def fun_last6_day(arr):
        try:
            arr=arr.sort_values()
            return (arr.iloc[-4] - arr.iloc[-5]).days
        except:
            return np.nan

    def fun_last7_day(arr):
        try:
            arr=arr.sort_values()
            return (arr.iloc[-4] - arr.iloc[-5]).days
        except:
            return np.nan     

    df_tr[df_tr['UID']==10012]

    df_tr[['UID','tr_channel','tr_day']].groupby(['UID','tr_channel']).agg(fun_last_day)

    df_tr[['UID','tr_channel','tr_time2']].groupby(['UID','tr_channel']).agg(fun_last2_se)

    df_tr_last_se=df_tr[['UID','tr_channel','tr_day','tr_time2']].groupby(['UID','tr_channel','tr_day'],as_index=False).agg(fun_last_se)

    df_tr_last_se=df_tr[['UID','tr_channel','tr_day','tr_time2']].groupby(['UID','tr_channel','tr_day'],as_index=False).agg(fun_last_se)
    df_tr_last_se2=pd.pivot_table(df_tr_last_se,index=['UID'],values=['tr_time2'],columns=['tr_channel'],aggfunc=['min','max'])



    #el df_tr_last_se3
    df_tr_last_se2.columns=['_'.join(('tr_last_se',str(k),i)) for i,j,k in df_tr_last_se2.columns]


    ############################################################ last sec per day

    df_tr_last_day_se=df_tr[['UID','tr_channel','tr_day','tr_time2']].groupby(['UID','tr_channel','tr_day'],as_index=False).agg(fun_last_se)
    df_tr_last_day_se2=pd.pivot_table(df_tr_last_day_se,index=['UID'],values=['tr_time2'],columns=['tr_channel'],aggfunc=['min','max','mean','median'])
    df_tr_last_day_se2.columns=['_'.join(('df_tr_last_day_se2',str(k),i)) for i,j,k in df_tr_last_day_se2.columns]

    df_tr_last2_day_se=df_tr[['UID','tr_channel','tr_day','tr_time2']].groupby(['UID','tr_channel','tr_day'],as_index=False).agg(fun_last2_se)
    df_tr_last2_day_se2=pd.pivot_table(df_tr_last2_day_se,index=['UID'],values=['tr_time2'],columns=['tr_channel'],aggfunc=['min','max','mean','median'])
    df_tr_last2_day_se2.columns=['_'.join(('df_tr_last2_day_se2',str(k),i)) for i,j,k in df_tr_last2_day_se2.columns]

    df_tr_last3_day_se=df_tr[['UID','tr_channel','tr_day','tr_time2']].groupby(['UID','tr_channel','tr_day'],as_index=False).agg(fun_last3_se)
    df_tr_last3_day_se2=pd.pivot_table(df_tr_last3_day_se,index=['UID'],values=['tr_time2'],columns=['tr_channel'],aggfunc=['min','max','mean','median'])
    df_tr_last3_day_se2.columns=['_'.join(('df_tr_last3_day_se2',str(k),i)) for i,j,k in df_tr_last3_day_se2.columns]

    ###
    df_tr_last4_day_se=df_tr[['UID','tr_channel','tr_day','tr_time2']].groupby(['UID','tr_channel','tr_day'],as_index=False).agg(fun_last4_se)
    df_tr_last4_day_se2=pd.pivot_table(df_tr_last4_day_se,index=['UID'],values=['tr_time2'],columns=['tr_channel'],aggfunc=['min','max','mean','median'])
    df_tr_last4_day_se2.columns=['_'.join(('df_tr_last4_day_se2',str(k),i)) for i,j,k in df_tr_last4_day_se2.columns]

    df_tr_last5_day_se=df_tr[['UID','tr_channel','tr_day','tr_time2']].groupby(['UID','tr_channel','tr_day'],as_index=False).agg(fun_last5_se)
    df_tr_last5_day_se2=pd.pivot_table(df_tr_last5_day_se,index=['UID'],values=['tr_time2'],columns=['tr_channel'],aggfunc=['min','max','mean','median'])
    df_tr_last5_day_se2.columns=['_'.join(('df_tr_last5_day_se2',str(k),i)) for i,j,k in df_tr_last5_day_se2.columns]

    df_tr_last6_day_se=df_tr[['UID','tr_channel','tr_day','tr_time2']].groupby(['UID','tr_channel','tr_day'],as_index=False).agg(fun_last6_se)
    df_tr_last6_day_se2=pd.pivot_table(df_tr_last6_day_se,index=['UID'],values=['tr_time2'],columns=['tr_channel'],aggfunc=['min','max','mean','median'])
    df_tr_last6_day_se2.columns=['_'.join(('df_tr_last6_day_se2',str(k),i)) for i,j,k in df_tr_last6_day_se2.columns]

    df_tr_last7_day_se=df_tr[['UID','tr_channel','tr_day','tr_time2']].groupby(['UID','tr_channel','tr_day'],as_index=False).agg(fun_last7_se)
    df_tr_last7_day_se2=pd.pivot_table(df_tr_last7_day_se,index=['UID'],values=['tr_time2'],columns=['tr_channel'],aggfunc=['min','max','mean','median'])
    df_tr_last7_day_se2.columns=['_'.join(('df_tr_last7_day_se2',str(k),i)) for i,j,k in df_tr_last7_day_se2.columns]



    #########################################################  last sec all day

    df_tr_last_se=df_tr[['UID','tr_channel','tr_time2']].groupby(['UID','tr_channel'],as_index=False).agg(fun_last_se)
    df_tr_last_se2=pd.pivot_table(df_tr_last_se,index=['UID'],values=['tr_time2'],columns=['tr_channel'])
    df_tr_last_se2.columns=['_'.join(('df_tr_last_se2',str(k))) for i,k in df_tr_last_se2.columns]

    df_tr_last2_se=df_tr[['UID','tr_channel','tr_time2']].groupby(['UID','tr_channel'],as_index=False).agg(fun_last2_se)
    df_tr_last2_se2=pd.pivot_table(df_tr_last2_se,index=['UID'],values=['tr_time2'],columns=['tr_channel'])
    df_tr_last2_se2.columns=['_'.join(('df_tr_last2_se2',str(k))) for i,k in df_tr_last2_se2.columns]

    df_tr_last3_se=df_tr[['UID','tr_channel','tr_time2']].groupby(['UID','tr_channel'],as_index=False).agg(fun_last3_se)
    df_tr_last3_se2=pd.pivot_table(df_tr_last3_se,index=['UID'],values=['tr_time2'],columns=['tr_channel'])
    df_tr_last3_se2.columns=['_'.join(('df_tr_last3_se2',str(k))) for i,k in df_tr_last3_se2.columns]

    ###
    df_tr_last4_se=df_tr[['UID','tr_channel','tr_time2']].groupby(['UID','tr_channel'],as_index=False).agg(fun_last4_se)
    df_tr_last4_se2=pd.pivot_table(df_tr_last4_se,index=['UID'],values=['tr_time2'],columns=['tr_channel'])
    df_tr_last4_se2.columns=['_'.join(('df_tr_last4_se2',str(k))) for i,k in df_tr_last4_se2.columns]

    df_tr_last5_se=df_tr[['UID','tr_channel','tr_time2']].groupby(['UID','tr_channel'],as_index=False).agg(fun_last5_se)
    df_tr_last5_se2=pd.pivot_table(df_tr_last5_se,index=['UID'],values=['tr_time2'],columns=['tr_channel'])
    df_tr_last5_se2.columns=['_'.join(('df_tr_last5_se2',str(k))) for i,k in df_tr_last5_se2.columns]

    df_tr_last6_se=df_tr[['UID','tr_channel','tr_time2']].groupby(['UID','tr_channel'],as_index=False).agg(fun_last6_se)
    df_tr_last6_se2=pd.pivot_table(df_tr_last6_se,index=['UID'],values=['tr_time2'],columns=['tr_channel'])
    df_tr_last6_se2.columns=['_'.join(('df_tr_last6_se2',str(k))) for i,k in df_tr_last6_se2.columns]

    df_tr_last7_se=df_tr[['UID','tr_channel','tr_time2']].groupby(['UID','tr_channel'],as_index=False).agg(fun_last7_se)
    df_tr_last7_se2=pd.pivot_table(df_tr_last7_se,index=['UID'],values=['tr_time2'],columns=['tr_channel'])
    df_tr_last7_se2.columns=['_'.join(('df_tr_last7_se2',str(k))) for i,k in df_tr_last7_se2.columns]


    ######################   last 1,2,3 day
    df_tr_last_day=df_tr[['UID','tr_channel','tr_time2']].groupby(['UID','tr_channel'],as_index=False).agg(fun_last_day)
    df_tr_last_day2=pd.pivot_table(df_tr_last_day,index=['UID'],values=['tr_time2'],columns=['tr_channel'])
    df_tr_last_day2.columns=['_'.join(('df_tr_last_day2',str(j))) for i,j in df_tr_last_day2.columns]

    df_tr_last2_day=df_tr[['UID','tr_channel','tr_time2']].groupby(['UID','tr_channel'],as_index=False).agg(fun_last2_day)
    df_tr_last2_day2=pd.pivot_table(df_tr_last2_day,index=['UID'],values=['tr_time2'],columns=['tr_channel'])
    df_tr_last2_day2.columns=['_'.join(('df_tr_last2_day2',str(j))) for i,j in df_tr_last2_day2.columns]

    df_tr_last3_day=df_tr[['UID','tr_channel','tr_time2']].groupby(['UID','tr_channel'],as_index=False).agg(fun_last3_day)
    df_tr_last3_day2=pd.pivot_table(df_tr_last3_day,index=['UID'],values=['tr_time2'],columns=['tr_channel'])
    df_tr_last3_day2.columns=['_'.join(('df_tr_last3_day2',str(j))) for i,j in df_tr_last3_day2.columns]

    ###
    df_tr_last4_day=df_tr[['UID','tr_channel','tr_time2']].groupby(['UID','tr_channel'],as_index=False).agg(fun_last4_day)
    df_tr_last4_day2=pd.pivot_table(df_tr_last4_day,index=['UID'],values=['tr_time2'],columns=['tr_channel'])
    df_tr_last4_day2.columns=['_'.join(('df_tr_last4_day2',str(j))) for i,j in df_tr_last4_day2.columns]

    df_tr_last5_day=df_tr[['UID','tr_channel','tr_time2']].groupby(['UID','tr_channel'],as_index=False).agg(fun_last5_day)
    df_tr_last5_day2=pd.pivot_table(df_tr_last5_day,index=['UID'],values=['tr_time2'],columns=['tr_channel'])
    df_tr_last5_day2.columns=['_'.join(('df_tr_last5_day2',str(j))) for i,j in df_tr_last5_day2.columns]

    df_tr_last6_day=df_tr[['UID','tr_channel','tr_time2']].groupby(['UID','tr_channel'],as_index=False).agg(fun_last6_day)
    df_tr_last6_day2=pd.pivot_table(df_tr_last6_day,index=['UID'],values=['tr_time2'],columns=['tr_channel'])
    df_tr_last6_day2.columns=['_'.join(('df_tr_last6_day2',str(j))) for i,j in df_tr_last6_day2.columns]

    df_tr_last7_day=df_tr[['UID','tr_channel','tr_time2']].groupby(['UID','tr_channel'],as_index=False).agg(fun_last7_day)
    df_tr_last7_day2=pd.pivot_table(df_tr_last7_day,index=['UID'],values=['tr_time2'],columns=['tr_channel'])
    df_tr_last7_day2.columns=['_'.join(('df_tr_last7_day2',str(j))) for i,j in df_tr_last7_day2.columns]
    
    ###
    df_tr_last_se3=df_tr[['UID','tr_time2']].groupby(['UID'],as_index=False).agg(fun_last_se)
    df_tr_last_se3.columns=['UID','tr_last_se3']

    df_tr_last2_se3=df_tr[['UID','tr_time2']].groupby(['UID'],as_index=False).agg(fun_last2_se)
    df_tr_last2_se3.columns=['UID','tr_last2_se3']

    df_tr_last3_se3=df_tr[['UID','tr_time2']].groupby(['UID'],as_index=False).agg(fun_last3_se)
    df_tr_last3_se3.columns=['UID','tr_last3_se3']

    df_tr_last4_se3=df_tr[['UID','tr_time2']].groupby(['UID'],as_index=False).agg(fun_last4_se)
    df_tr_last4_se3.columns=['UID','tr_last4_se3']

    df_tr_last5_se3=df_tr[['UID','tr_time2']].groupby(['UID'],as_index=False).agg(fun_last5_se)
    df_tr_last5_se3.columns=['UID','tr_last5_se3']

    df_tr_last6_se3=df_tr[['UID','tr_time2']].groupby(['UID'],as_index=False).agg(fun_last6_se)
    df_tr_last6_se3.columns=['UID','tr_last6_se3']

    df_tr_last7_se3=df_tr[['UID','tr_time2']].groupby(['UID'],as_index=False).agg(fun_last7_se)
    df_tr_last7_se3.columns=['UID','tr_last7_se3']


    data_train6=data_train5
    
    print('---data_train6-- ')
    print(data_train6.shape)



    data_train6=data_train6.merge(df_tr_last_day_se2,how='left',on='UID')
    data_train6=data_train6.merge(df_tr_last2_day_se2,how='left',on='UID')
    data_train6=data_train6.merge(df_tr_last3_day_se2,how='left',on='UID')
    data_train6=data_train6.merge(df_tr_last4_day_se2,how='left',on='UID')
    data_train6=data_train6.merge(df_tr_last5_day_se2,how='left',on='UID')
    data_train6=data_train6.merge(df_tr_last6_day_se2,how='left',on='UID')
    data_train6=data_train6.merge(df_tr_last7_day_se2,how='left',on='UID')

    data_train6=data_train6.merge(df_tr_last_se2,how='left',on='UID')
    data_train6=data_train6.merge(df_tr_last2_se2,how='left',on='UID')
    data_train6=data_train6.merge(df_tr_last3_se2,how='left',on='UID')
    data_train6=data_train6.merge(df_tr_last4_se2,how='left',on='UID')
    data_train6=data_train6.merge(df_tr_last5_se2,how='left',on='UID')
    data_train6=data_train6.merge(df_tr_last6_se2,how='left',on='UID')
    data_train6=data_train6.merge(df_tr_last7_se2,how='left',on='UID')

    data_train6=data_train6.merge(df_tr_last_day2,how='left',on='UID')
    data_train6=data_train6.merge(df_tr_last2_day2,how='left',on='UID')
    data_train6=data_train6.merge(df_tr_last3_day2,how='left',on='UID')
    data_train6=data_train6.merge(df_tr_last4_day2,how='left',on='UID')
    data_train6=data_train6.merge(df_tr_last5_day2,how='left',on='UID')
    data_train6=data_train6.merge(df_tr_last6_day2,how='left',on='UID')
    data_train6=data_train6.merge(df_tr_last7_day2,how='left',on='UID')
    
    data_train6=data_train6.merge(df_tr_last_se3,how='left',on='UID')
    data_train6=data_train6.merge(df_tr_last2_se3,how='left',on='UID')
    data_train6=data_train6.merge(df_tr_last3_se3,how='left',on='UID')
    data_train6=data_train6.merge(df_tr_last4_se3,how='left',on='UID')
    data_train6=data_train6.merge(df_tr_last5_se3,how='left',on='UID')
    data_train6=data_train6.merge(df_tr_last6_se3,how='left',on='UID')
    data_train6=data_train6.merge(df_tr_last7_se3,how='left',on='UID')

    print('---data_train6-- ')
    print(data_train6.shape)

    df_op['day']='2017-10-'+df_op['day'].apply(lambda x:str(x) if x>=10 else '0'+str(x))



    df_op['time1']=df_op['day']+' '+df_op['time']
    df_op['time2']=df_op['time1'].apply(lambda x:pd.to_datetime(x,format='%Y-%m-%d %H:%M:%S'))



    df_op.info(memory_usage='deep')

    df_op['op_day']=df_op['time2'].apply(lambda x:x.day)

    ############################################################ last sec per day

    df_op_last_day_se=df_op[['UID','success','op_day','time2']].groupby(['UID','success','op_day'],as_index=False).agg(fun_last_se)
    df_op_last_day_se2=pd.pivot_table(df_op_last_day_se,index=['UID'],values=['time2'],columns=['success'],aggfunc=['min','max','mean','median'])
    df_op_last_day_se2.columns=['_'.join(('df_op_last_day_se2',str(k),i)) for i,j,k in df_op_last_day_se2.columns]

    df_op_last2_day_se=df_op[['UID','success','op_day','time2']].groupby(['UID','success','op_day'],as_index=False).agg(fun_last2_se)
    df_op_last2_day_se2=pd.pivot_table(df_op_last2_day_se,index=['UID'],values=['time2'],columns=['success'],aggfunc=['min','max','mean','median'])
    df_op_last2_day_se2.columns=['_'.join(('df_op_last2_day_se2',str(k),i)) for i,j,k in df_op_last2_day_se2.columns]

    df_op_last3_day_se=df_op[['UID','success','op_day','time2']].groupby(['UID','success','op_day'],as_index=False).agg(fun_last3_se)
    df_op_last3_day_se2=pd.pivot_table(df_op_last3_day_se,index=['UID'],values=['time2'],columns=['success'],aggfunc=['min','max','mean','median'])
    df_op_last3_day_se2.columns=['_'.join(('df_op_last3_day_se2',str(k),i)) for i,j,k in df_op_last3_day_se2.columns]

    df_op_last4_day_se=df_op[['UID','success','op_day','time2']].groupby(['UID','success','op_day'],as_index=False).agg(fun_last4_se)
    df_op_last4_day_se2=pd.pivot_table(df_op_last4_day_se,index=['UID'],values=['time2'],columns=['success'],aggfunc=['min','max','mean','median'])
    df_op_last4_day_se2.columns=['_'.join(('df_op_last4_day_se2',str(k),i)) for i,j,k in df_op_last4_day_se2.columns]

    df_op_last5_day_se=df_op[['UID','success','op_day','time2']].groupby(['UID','success','op_day'],as_index=False).agg(fun_last5_se)
    df_op_last5_day_se2=pd.pivot_table(df_op_last5_day_se,index=['UID'],values=['time2'],columns=['success'],aggfunc=['min','max','mean','median'])
    df_op_last5_day_se2.columns=['_'.join(('df_op_last5_day_se2',str(k),i)) for i,j,k in df_op_last5_day_se2.columns]

    df_op_last6_day_se=df_op[['UID','success','op_day','time2']].groupby(['UID','success','op_day'],as_index=False).agg(fun_last6_se)
    df_op_last6_day_se2=pd.pivot_table(df_op_last6_day_se,index=['UID'],values=['time2'],columns=['success'],aggfunc=['min','max','mean','median'])
    df_op_last6_day_se2.columns=['_'.join(('df_op_last6_day_se2',str(k),i)) for i,j,k in df_op_last6_day_se2.columns]

    df_op_last7_day_se=df_op[['UID','success','op_day','time2']].groupby(['UID','success','op_day'],as_index=False).agg(fun_last7_se)
    df_op_last7_day_se2=pd.pivot_table(df_op_last7_day_se,index=['UID'],values=['time2'],columns=['success'],aggfunc=['min','max','mean','median'])
    df_op_last7_day_se2.columns=['_'.join(('df_op_last7_day_se2',str(k),i)) for i,j,k in df_op_last7_day_se2.columns]


    #########################################################  last sec all day

    df_op_last_se=df_op[['UID','success','time2']].groupby(['UID','success'],as_index=False).agg(fun_last_se)
    df_op_last_se2=pd.pivot_table(df_op_last_se,index=['UID'],values=['time2'],columns=['success'])
    df_op_last_se2.columns=['_'.join(('df_op_last_se2',str(k))) for i,k in df_op_last_se2.columns]

    df_op_last2_se=df_op[['UID','success','time2']].groupby(['UID','success'],as_index=False).agg(fun_last2_se)
    df_op_last2_se2=pd.pivot_table(df_op_last2_se,index=['UID'],values=['time2'],columns=['success'])
    df_op_last2_se2.columns=['_'.join(('df_op_last2_se2',str(k))) for i,k in df_op_last2_se2.columns]

    df_op_last3_se=df_op[['UID','success','time2']].groupby(['UID','success'],as_index=False).agg(fun_last3_se)
    df_op_last3_se2=pd.pivot_table(df_op_last3_se,index=['UID'],values=['time2'],columns=['success'])
    df_op_last3_se2.columns=['_'.join(('df_op_last3_se2',str(k))) for i,k in df_op_last3_se2.columns]
    ###
    df_op_last4_se=df_op[['UID','success','time2']].groupby(['UID','success'],as_index=False).agg(fun_last4_se)
    df_op_last4_se2=pd.pivot_table(df_op_last4_se,index=['UID'],values=['time2'],columns=['success'])
    df_op_last4_se2.columns=['_'.join(('df_op_last4_se2',str(k))) for i,k in df_op_last4_se2.columns]

    df_op_last5_se=df_op[['UID','success','time2']].groupby(['UID','success'],as_index=False).agg(fun_last5_se)
    df_op_last5_se2=pd.pivot_table(df_op_last5_se,index=['UID'],values=['time2'],columns=['success'])
    df_op_last5_se2.columns=['_'.join(('df_op_last5_se2',str(k))) for i,k in df_op_last5_se2.columns]

    df_op_last6_se=df_op[['UID','success','time2']].groupby(['UID','success'],as_index=False).agg(fun_last6_se)
    df_op_last6_se2=pd.pivot_table(df_op_last6_se,index=['UID'],values=['time2'],columns=['success'])
    df_op_last6_se2.columns=['_'.join(('df_op_last6_se2',str(k))) for i,k in df_op_last6_se2.columns]

    df_op_last7_se=df_op[['UID','success','time2']].groupby(['UID','success'],as_index=False).agg(fun_last7_se)
    df_op_last7_se2=pd.pivot_table(df_op_last7_se,index=['UID'],values=['time2'],columns=['success'])
    df_op_last7_se2.columns=['_'.join(('df_op_last7_se2',str(k))) for i,k in df_op_last7_se2.columns]
######################   last 1,2,3 day

    df_op_last_day=df_op[['UID','success','time2']].groupby(['UID','success'],as_index=False).agg(fun_last_day)
    df_op_last_day2=pd.pivot_table(df_op_last_day,index=['UID'],values=['time2'],columns=['success'])
    df_op_last_day2.columns=['_'.join(('df_op_last_day2',str(j))) for i,j in df_op_last_day2.columns]

    df_op_last2_day=df_op[['UID','success','time2']].groupby(['UID','success'],as_index=False).agg(fun_last2_day)
    df_op_last2_day2=pd.pivot_table(df_op_last2_day,index=['UID'],values=['time2'],columns=['success'])
    df_op_last2_day2.columns=['_'.join(('df_op_last2_day2',str(j))) for i,j in df_op_last2_day2.columns]

    df_op_last3_day=df_op[['UID','success','time2']].groupby(['UID','success'],as_index=False).agg(fun_last3_day)
    df_op_last3_day2=pd.pivot_table(df_op_last3_day,index=['UID'],values=['time2'],columns=['success'])
    df_op_last3_day2.columns=['_'.join(('df_op_last3_day2',str(j))) for i,j in df_op_last3_day2.columns]
    ###
    df_op_last4_day=df_op[['UID','success','time2']].groupby(['UID','success'],as_index=False).agg(fun_last4_day)
    df_op_last4_day2=pd.pivot_table(df_op_last4_day,index=['UID'],values=['time2'],columns=['success'])
    df_op_last4_day2.columns=['_'.join(('df_op_last4_day2',str(j))) for i,j in df_op_last4_day2.columns]

    df_op_last5_day=df_op[['UID','success','time2']].groupby(['UID','success'],as_index=False).agg(fun_last5_day)
    df_op_last5_day2=pd.pivot_table(df_op_last5_day,index=['UID'],values=['time2'],columns=['success'])
    df_op_last5_day2.columns=['_'.join(('df_op_last5_day2',str(j))) for i,j in df_op_last5_day2.columns]

    df_op_last6_day=df_op[['UID','success','time2']].groupby(['UID','success'],as_index=False).agg(fun_last6_day)
    df_op_last6_day2=pd.pivot_table(df_op_last6_day,index=['UID'],values=['time2'],columns=['success'])
    df_op_last6_day2.columns=['_'.join(('df_op_last6_day2',str(j))) for i,j in df_op_last6_day2.columns]

    df_op_last7_day=df_op[['UID','success','time2']].groupby(['UID','success'],as_index=False).agg(fun_last7_day)
    df_op_last7_day2=pd.pivot_table(df_op_last7_day,index=['UID'],values=['time2'],columns=['success'])
    df_op_last7_day2.columns=['_'.join(('df_op_last7_day2',str(j))) for i,j in df_op_last7_day2.columns]

####################################### op last1,2,3 sec not day
    df_op_last_se3=df_op[['UID','time2']].groupby(['UID'],as_index=False).agg(fun_last_se)
    df_op_last_se3.columns=['UID','op_last_se3']

    df_op_last2_se3=df_op[['UID','time2']].groupby(['UID'],as_index=False).agg(fun_last2_se)
    df_op_last2_se3.columns=['UID','op_last2_se3']

    df_op_last3_se3=df_op[['UID','time2']].groupby(['UID'],as_index=False).agg(fun_last3_se)
    df_op_last3_se3.columns=['UID','op_last3_se3']
    ###
    df_op_last4_se3=df_op[['UID','time2']].groupby(['UID'],as_index=False).agg(fun_last4_se)
    df_op_last4_se3.columns=['UID','op_last4_se3']

    df_op_last5_se3=df_op[['UID','time2']].groupby(['UID'],as_index=False).agg(fun_last5_se)
    df_op_last5_se3.columns=['UID','op_last5_se3']

    df_op_last6_se3=df_op[['UID','time2']].groupby(['UID'],as_index=False).agg(fun_last6_se)
    df_op_last6_se3.columns=['UID','op_last6_se3']

    df_op_last7_se3=df_op[['UID','time2']].groupby(['UID'],as_index=False).agg(fun_last7_se)
    df_op_last7_se3.columns=['UID','op_last7_se3']



    data_train7=data_train6.copy()
    data_train7=data_train7.merge(df_op_last_day_se2,how='left',on='UID')
    data_train7=data_train7.merge(df_op_last2_day_se2,how='left',on='UID')
    data_train7=data_train7.merge(df_op_last3_day_se2,how='left',on='UID')
    data_train7=data_train7.merge(df_op_last4_day_se2,how='left',on='UID')
    data_train7=data_train7.merge(df_op_last5_day_se2,how='left',on='UID')
    data_train7=data_train7.merge(df_op_last6_day_se2,how='left',on='UID')
    data_train7=data_train7.merge(df_op_last7_day_se2,how='left',on='UID')

    data_train7=data_train7.merge(df_op_last_se2,how='left',on='UID')
    data_train7=data_train7.merge(df_op_last2_se2,how='left',on='UID')
    data_train7=data_train7.merge(df_op_last3_se2,how='left',on='UID')
    data_train7=data_train7.merge(df_op_last4_se2,how='left',on='UID')
    data_train7=data_train7.merge(df_op_last5_se2,how='left',on='UID')
    data_train7=data_train7.merge(df_op_last6_se2,how='left',on='UID')
    data_train7=data_train7.merge(df_op_last7_se2,how='left',on='UID')


    data_train7=data_train7.merge(df_op_last_day2,how='left',on='UID')
    data_train7=data_train7.merge(df_op_last2_day2,how='left',on='UID')
    data_train7=data_train7.merge(df_op_last3_day2,how='left',on='UID')
    data_train7=data_train7.merge(df_op_last4_day2,how='left',on='UID')
    data_train7=data_train7.merge(df_op_last5_day2,how='left',on='UID')
    data_train7=data_train7.merge(df_op_last6_day2,how='left',on='UID')
    data_train7=data_train7.merge(df_op_last7_day2,how='left',on='UID')
    
    data_train7=data_train7.merge(df_op_last_se3,how='left',on='UID')
    data_train7=data_train7.merge(df_op_last2_se3,how='left',on='UID')
    data_train7=data_train7.merge(df_op_last3_se3,how='left',on='UID')
    data_train7=data_train7.merge(df_op_last4_se3,how='left',on='UID')
    data_train7=data_train7.merge(df_op_last5_se3,how='left',on='UID')
    data_train7=data_train7.merge(df_op_last6_se3,how='left',on='UID')
    data_train7=data_train7.merge(df_op_last7_se3,how='left',on='UID')

    data_train8=data_train7.drop_duplicates()
    data_train8.fillna(-999,inplace=True)
    
    return data_train8



In [3]:
data_train8=process_data()

D:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2850: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:544: UserWarning: merging betwee

---data_train6-- 
(62377, 241)
---data_train6-- 
(62377, 442)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2812383 entries, 0 to 1769047
Data columns (total 22 columns):
UID             int64
day             object
mode            object
success         object
time            object
os              int64
version         object
device1         object
device2         object
device_code1    object
device_code2    object
device_code3    object
mac1            object
mac2            object
ip1             object
ip2             object
wifi            object
geo_code        object
ip1_sub         object
ip2_sub         object
time1           object
time2           datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(19)
memory usage: 3.5 GB


In [4]:
data_train8.shape

(62377, 562)

In [5]:
data_train8.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62377 entries, 0 to 62376
Columns: 562 entries, UID to op_last7_se3
dtypes: float64(559), int64(3)
memory usage: 267.9 MB


In [6]:
#df_tag1=pd.read_csv('F:\study\\ml\\DataSet\\TianChengJinRong\\tag_TRAIN.csv')
df_tag1=pd.read_csv('F:\\study\\ml\\DataSet\\TianChengJinRong\\round1-new\\tag_train_new.csv')

In [7]:
df_tag1.shape

(31179, 2)

In [8]:
data_train8.shape

(62377, 562)

In [9]:
data_train8=data_train8.merge(df_tag1,how='left',on='UID')

In [10]:
data_train8.shape

(62377, 563)

In [11]:
data_train8.head()

,UID,op_count,tr_count,op_day_1,op_day_2,op_day_3,op_day_4,op_day_5,op_day_6,op_day_7,op_day_8,op_day_9,op_day_10,op_day_11,op_day_12,...,df_op_last5_day2_None,df_op_last6_day2_0.0,df_op_last6_day2_1.0,df_op_last6_day2_None,df_op_last7_day2_0.0,df_op_last7_day2_1.0,df_op_last7_day2_None,op_last_se3,op_last2_se3,op_last3_se3,op_last4_se3,op_last5_se3,op_last6_se3,op_last7_se3,Tag
0,10000,5,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-999.0,-999.0,0.0,-999.0,-999.0,0.0,-999.0,20.0,13.0,79249.0,123.0,-999.0,-999.0,-999.0,1.0
1,10001,65,18,0.0,11.0,14.0,5.0,4.0,0.0,8.0,0.0,0.0,0.0,12.0,4.0,...,-999.0,-999.0,0.0,-999.0,-999.0,0.0,-999.0,88.0,12.0,1.0,57.0,9.0,2.0,8666.0,0.0
2,10002,11,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-999.0,-999.0,0.0,-999.0,-999.0,0.0,-999.0,20.0,47.0,0.0,35.0,89.0,22.0,38.0,0.0
3,10003,14,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-999.0,-999.0,0.0,-999.0,-999.0,0.0,-999.0,22.0,55.0,24.0,95.0,41.0,1041.0,349.0,0.0
4,10004,29,7,3.0,0.0,3.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,...,-999.0,-999.0,2.0,-999.0,-999.0,2.0,-999.0,2.0,69919.0,3.0,7560.0,154.0,0.0,0.0,0.0


In [33]:
data_train9=data_train8[data_train8.Tag.notna()]
data_test9=data_train8[data_train8.Tag.isna()]
X_data=data_train9.iloc[:,1:-1]
y_data=data_train9.iloc[:,-1]

In [34]:
data_train9.shape

(31179, 563)

In [35]:
data_test9.shape

(31198, 563)

In [36]:
df_tag1.shape

(31179, 2)

In [37]:
data_train9=data_train9.drop_duplicates()
data_train9=data_train9.fillna(-999)
data_test9=data_test9.drop_duplicates()
data_test9=data_test9.fillna(-999)

In [17]:
from sklearn.externals import joblib
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from sklearn.cross_validation import train_test_split,cross_val_score
import lightgbm as lgb

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [18]:
X_train,X_test,y_train,y_test=train_test_split(data_train9.iloc[:,1:-1],data_train9.iloc[:,-1],test_size=0.1,random_state=13)

In [19]:
data_train9.head()

,UID,op_count,tr_count,op_day_1,op_day_2,op_day_3,op_day_4,op_day_5,op_day_6,op_day_7,op_day_8,op_day_9,op_day_10,op_day_11,op_day_12,...,df_op_last5_day2_None,df_op_last6_day2_0.0,df_op_last6_day2_1.0,df_op_last6_day2_None,df_op_last7_day2_0.0,df_op_last7_day2_1.0,df_op_last7_day2_None,op_last_se3,op_last2_se3,op_last3_se3,op_last4_se3,op_last5_se3,op_last6_se3,op_last7_se3,Tag
0,10000,5,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-999.0,-999.0,0.0,-999.0,-999.0,0.0,-999.0,20.0,13.0,79249.0,123.0,-999.0,-999.0,-999.0,1.0
1,10001,65,18,0.0,11.0,14.0,5.0,4.0,0.0,8.0,0.0,0.0,0.0,12.0,4.0,...,-999.0,-999.0,0.0,-999.0,-999.0,0.0,-999.0,88.0,12.0,1.0,57.0,9.0,2.0,8666.0,0.0
2,10002,11,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-999.0,-999.0,0.0,-999.0,-999.0,0.0,-999.0,20.0,47.0,0.0,35.0,89.0,22.0,38.0,0.0
3,10003,14,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-999.0,-999.0,0.0,-999.0,-999.0,0.0,-999.0,22.0,55.0,24.0,95.0,41.0,1041.0,349.0,0.0
4,10004,29,7,3.0,0.0,3.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,...,-999.0,-999.0,2.0,-999.0,-999.0,2.0,-999.0,2.0,69919.0,3.0,7560.0,154.0,0.0,0.0,0.0


In [20]:
# X_train.head()

y_train.head()

41165    0.0
8073     1.0
29216    0.0
10265    1.0
37496    1.0
Name: Tag, dtype: float64

In [21]:
X_train.shape

(28061, 561)

In [22]:
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.model_selection import GridSearchCV

In [23]:
model44= GridSearchCV(
    estimator=lgb.LGBMClassifier(is_unbalance=True,),
    param_grid={
        'num_leaves':[114],
        'n_estimators': [1000],
        'learning_rate': [0.064],
        'max_depth': [3],
        #'subsample': [1],
         'subsample': [0.8],
        'colsample_bytree': [0.56],
#         'scale_pos_weight': [2.5, ]
#         'is_unbalance':True
    },
    scoring='f1',
    cv=3,
    n_jobs=4,
    verbose=1).fit(X_train,y_train)

ypred44=model44.predict(X_test)
predictions=ypred44
print('准确率',accuracy_score(y_test,predictions))
print('混淆矩阵',confusion_matrix(y_test,predictions))
print('分类报告',classification_report(y_test,predictions))

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=4)]: Done   3 out of   3 | elapsed:   29.9s finished


准确率 0.9355355997434253
混淆矩阵 [[2556  122]
 [  79  361]]
分类报告              precision    recall  f1-score   support

        0.0       0.97      0.95      0.96      2678
        1.0       0.75      0.82      0.78       440

avg / total       0.94      0.94      0.94      3118



D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [24]:
from sklearn.metrics import make_scorer
def tpr_weight_funtion(y_true,predict):
    y_predict=predict[:,1]
    d = pd.DataFrame()
    d['prob'] = list(y_predict)
    d['y'] = list(y_true)
    d = d.sort_values(['prob'], ascending=[0])
    y = d.y
    PosAll = pd.Series(y).value_counts()[1]
    NegAll = pd.Series(y).value_counts()[0]
    pCumsum = d['y'].cumsum()
    nCumsum = np.arange(len(y)) - pCumsum + 1
    pCumsumPer = pCumsum / PosAll
    nCumsumPer = nCumsum / NegAll
    TR1 = pCumsumPer[abs(nCumsumPer-0.001).idxmin()]
    TR2 = pCumsumPer[abs(nCumsumPer-0.005).idxmin()]
    TR3 = pCumsumPer[abs(nCumsumPer-0.01).idxmin()]
    return 0.4 * TR1 + 0.3 * TR2 + 0.3 * TR3

tpr_weight_scorer = make_scorer(tpr_weight_funtion, greater_is_better=True, needs_proba=True)

In [25]:
#use tpe to find best param
from hyperopt import  hp,fmin,space_eval,tpe
from sklearn.model_selection import cross_val_score,StratifiedKFold

g_i=0
def objective(params):
    global g_i 
    g_i+=1
    params={
         'num_leaves':int(params['num_leaves']),
         #'max_depth':int(params['max_depth']),
         'colsample_bytree':'{:.3f}'.format(params['colsample_bytree']),
         'learning_rate':'{:.3f}'.format(params['learning_rate']),
         'subsample':'{:.3f}'.format(params['subsample']),
    }
    
    clf=lgb.LGBMClassifier(
        n_estimators=500,
        is_unbalance=True,
        n_jobs=3,
        **params
    )
    
    score=-cross_val_score(clf,X_train,y_train,scoring=tpr_weight_scorer,cv=StratifiedKFold()).mean()
    print('------- {} --------'.format(g_i))
    print('trp weight: {:.5f}'.format(score))
    return score


space={
    'num_leaves':hp.quniform('num_leaves',8,256,4),
    #'max_depth':hp.quniform('max_depth',2,10,1),
    'colsample_bytree':hp.uniform('colsample_bytree',0.5,1),
    'learning_rate':hp.uniform('learning_rate',0.001,0.1),
     'subsample':hp.uniform('subsample',0.5,1),
}

best=fmin(fn=objective,
         space=space,
         algo=tpe.suggest,
         max_evals=10)

------- 1 --------
trp weight: -0.72905
------- 2 --------
trp weight: -0.73077
------- 3 --------
trp weight: -0.68835
------- 4 --------
trp weight: -0.72929
------- 5 --------
trp weight: -0.73023
------- 6 --------
trp weight: -0.72122
------- 7 --------
trp weight: -0.72877
------- 8 --------
trp weight: -0.72133
------- 9 --------
trp weight: -0.73067
------- 10 --------
trp weight: -0.72822


In [26]:
print('Hyperopt estimated optimum {}'.format(best))

Hyperopt estimated optimum {'colsample_bytree': 0.6616371097819339, 'learning_rate': 0.09988243540352636, 'num_leaves': 164.0, 'subsample': 0.9708411018626875}


In [27]:
# X_train1,X_test1,y_train1,y_test1=train_test_split(data_train9.iloc[:,1:-1],data_train9.iloc[:,-1],test_size=0.5,random_state=13)

lgbm=lgb.LGBMClassifier(
    colsample_bytree= 0.9853628094788913, 
    learning_rate= 0.055652975453383, 
    #max_depth= 6,
    num_leaves= 1720, 
    subsample= 0.6960563467153605,
    is_unbalance=True,
    n_estimators=500).fit(X_train,y_train)

predictions=lgbm.predict(X_test)
print('准确率',accuracy_score(y_test,predictions))
print('混淆矩阵',confusion_matrix(y_test,predictions))
print('分类报告',classification_report(y_test,predictions))

准确率 0.9618345093008339
混淆矩阵 [[2666   12]
 [ 107  333]]
分类报告              precision    recall  f1-score   support

        0.0       0.96      1.00      0.98      2678
        1.0       0.97      0.76      0.85       440

avg / total       0.96      0.96      0.96      3118



D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [28]:
df_fi=pd.DataFrame()
df_fi['name']=X_train.columns.values
df_fi['importance']=lgbm.feature_importances_
df_fi.sort_values(by=['importance'],axis=0,ascending=False)

,name,importance
233,tr_trans_amt_max,8236
232,tr_trans_amt_min,8165
234,tr_trans_amt_mean,7945
97,mode_0,5763
445,df_op_last_day_se2_1.0_max,4915
442,df_op_last_day_se2_1.0_min,4846
235,tr_trans_amt_median,4708
451,df_op_last_day_se2_1.0_median,4524
457,df_op_last2_day_se2_1.0_max,4517
45,version_0,4470


In [29]:
df_fi_col=df_fi[df_fi['importance']>0]['name'].values

In [30]:
#use tpe to find best param
from hyperopt import  hp,fmin,space_eval,tpe
from sklearn.model_selection import cross_val_score,StratifiedKFold

g_i=0
def objective2(params):
    global g_i 
    g_i+=1
    params={
         'num_leaves':int(params['num_leaves']),
         #'max_depth':int(params['max_depth']),
         'colsample_bytree':'{:.3f}'.format(params['colsample_bytree']),
         'learning_rate':'{:.3f}'.format(params['learning_rate']),
         'subsample':'{:.3f}'.format(params['subsample']),
    }
    
    clf=lgb.LGBMClassifier(
        n_estimators=500,
        is_unbalance=True,
        n_jobs=3,
        **params
    )
    
    score=-cross_val_score(clf,X_train[df_fi_col],y_train,scoring=tpr_weight_scorer,cv=StratifiedKFold()).mean()
    print('------- {} --------'.format(g_i))
    print('trp weight: {:.5f}'.format(score))
    return score


space2={
    'num_leaves':hp.quniform('num_leaves',8,256,4),
    #'max_depth':hp.quniform('max_depth',2,10,1),
    'colsample_bytree':hp.uniform('colsample_bytree',0.5,1),
    'learning_rate':hp.uniform('learning_rate',0.001,0.1),
     'subsample':hp.uniform('subsample',0.5,1),
}

best2=fmin(fn=objective2,
         space=space2,
         algo=tpe.suggest,
         max_evals=10)

    

------- 1 --------
trp weight: -0.72952
------- 2 --------
trp weight: -0.69852
------- 3 --------
trp weight: -0.72809
------- 4 --------
trp weight: -0.64140
------- 5 --------
trp weight: -0.71883
------- 6 --------
trp weight: -0.72781
------- 7 --------
trp weight: -0.72851
------- 8 --------
trp weight: -0.72853
------- 9 --------
trp weight: -0.72835
------- 10 --------
trp weight: -0.72874


In [31]:
print(best2)

{'colsample_bytree': 0.7630976701039046, 'learning_rate': 0.0780067863137393, 'num_leaves': 136.0, 'subsample': 0.5084624573034091}


In [32]:
X_data.shape

NameError: name 'X_data' is not defined

In [ ]:
X_train.shape

In [ ]:
#use tpe to find best param
from hyperopt import  hp,fmin,space_eval,tpe
from sklearn.model_selection import cross_val_score,StratifiedKFold

X_data=data_train9.iloc[:,1:-1]
y_data=data_train9.iloc[:,-1]

g_i=0
def objective3(params):
    global g_i 
    g_i+=1
    params={
         'num_leaves':int(params['num_leaves']),
         #'max_depth':int(params['max_depth']),
         'colsample_bytree':'{:.3f}'.format(params['colsample_bytree']),
         'learning_rate':'{:.3f}'.format(params['learning_rate']),
         'subsample':'{:.3f}'.format(params['subsample']),
    }
    
    clf=lgb.LGBMClassifier(
        n_estimators=500,
        is_unbalance=True,
        n_jobs=3,
        **params
    )
    
    score=-cross_val_score(clf,X_data[df_fi_col],y_data,scoring=tpr_weight_scorer,cv=StratifiedKFold()).mean()
    print('------- {} --------'.format(g_i))
    print('trp weight: {:.5f}'.format(score))
    return score


space3={
    'num_leaves':hp.quniform('num_leaves',8,512,4),
    #'max_depth':hp.quniform('max_depth',2,10,1),
    'colsample_bytree':hp.uniform('colsample_bytree',0.5,1),
    'learning_rate':hp.uniform('learning_rate',0.001,0.1),
     'subsample':hp.uniform('subsample',0.5,1),
}

best3=fmin(fn=objective3,
         space=space3,
         algo=tpe.suggest,
         max_evals=10)

    

In [ ]:
################### for all Train data
###  data_train9.iloc[:,1:-1],data_train9.iloc[:,-1]
X_data=data_train9.iloc[:,1:-1]
y_data=data_train9.iloc[:,-1]



#use tpe to find best param
from hyperopt import  hp,fmin,space_eval,tpe
from sklearn.model_selection import cross_val_score,StratifiedKFold

obj_call_count = 0
def objective(params):
    global obj_call_count
    obj_call_count += 1
    params={
          'num_leaves':int(params['num_leaves']),
          'min_data_in_leaf':int(params['min_data_in_leaf']),
         # 'max_depth':int(params['max_depth']),
          'colsample_bytree':'{:.4f}'.format(params['colsample_bytree']),
          'learning_rate':'{:.4f}'.format(params['learning_rate']),
          'subsample':'{:.4f}'.format(params['subsample']),
          'min_sum_hessian_in_leaf':'{:.4f}'.format(params['min_sum_hessian_in_leaf']),
          'max_bin':int(params['max_bin']),
          'bagging_freq':int(params['bagging_freq']),
          'lambda_l1':'{:.4f}'.format(params['lambda_l1']),
          'lambda_l2':'{:.4f}'.format(params['lambda_l2']),
          'n_estimators' :int(params['n_estimators'])
        
    }
    
    clf=lgb.LGBMClassifier(
        #n_estimators=500,
        is_unbalance=True,
        n_jobs=3,
        **params
    )
    
    score=-cross_val_score(clf,X_data,y_data,scoring=tpr_weight_scorer,cv=StratifiedKFold()).mean()
    print('---------------{}-----------------------'.format(obj_call_count))
    print('----tpe wieght: {:.3f}'.format(score))
    #print(params)
    print('-----------------------------------------')
    return score


space={
    'num_leaves':hp.quniform('num_leaves',8,2048,4),
    'min_data_in_leaf':  hp.quniform ('min_data_in_leaf', 10, 100, 10),
   # 'max_depth':hp.quniform('max_depth',2,30,1),
    'colsample_bytree':hp.uniform('colsample_bytree',0.3,1),
    'learning_rate':hp.uniform('learning_rate',0.001,0.5),
    'subsample':hp.uniform('subsample',0.3,1),
    'min_sum_hessian_in_leaf': hp.loguniform('min_sum_hessian_in_leaf', 0, 2.3),
    'max_bin': hp.quniform ('max_bin', 128, 2048, 1),
   'bagging_freq': hp.quniform ('bagging_freq', 1, 10, 1),
    'lambda_l1': hp.uniform('lambda_l1', 0, 10 ),
    'lambda_l2': hp.uniform('lambda_l2', 0, 10 ),
    'n_estimators':hp.quniform('n_estimators',100,1000,100),
}

best=fmin(fn=objective,
         space=space,
         algo=tpe.suggest,
         max_evals=500)

    

In [98]:
# df_fi.loc[df_fi.name in df_fi_mode_col,:]
df_fi[df_fi['name'].isin(df_fi_mode_col)].sort_values(by=['importance'],ascending=False)

,name,importance
97,mode_0,939
102,mode_5,844
100,mode_3,350
108,mode_11,343
104,mode_7,269
105,mode_8,248
119,mode_22,155
126,mode_29,151
113,mode_16,131
112,mode_15,122


In [65]:
# X_train1,X_test1,y_train1,y_test1=train_test_split(data_train9.iloc[:,1:-1],data_train9.iloc[:,-1],test_size=0.5,random_state=13)

lgbm=lgb.LGBMClassifier(
    colsample_bytree= 0.5716757426190645, 
    learning_rate= 0.09836162315529416, 
    max_depth= 6,
    num_leaves= 64, 
    subsample= 0.5925294416956224,
    is_unbalance=True,
    n_estimators=500).fit(X_train[df_fi_col],y_train)

predictions=lgbm.predict(X_test[df_fi_col])
print('准确率',accuracy_score(y_test,predictions))
print('混淆矩阵',confusion_matrix(y_test,predictions))
print('分类报告',classification_report(y_test,predictions))

准确率 0.9573444515715203
混淆矩阵 [[5292   86]
 [ 180  678]]
分类报告              precision    recall  f1-score   support

        0.0       0.97      0.98      0.98      5378
        1.0       0.89      0.79      0.84       858

avg / total       0.96      0.96      0.96      6236



D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [66]:
def tpr_weight_funtion1(y_true,y_predict):
    #y_predict=predict[:,1]
    d = pd.DataFrame()
    d['prob'] = list(y_predict)
    d['y'] = list(y_true)
    d = d.sort_values(['prob'], ascending=[0])
    y = d.y
    PosAll = pd.Series(y).value_counts()[1]
    NegAll = pd.Series(y).value_counts()[0]
    pCumsum = d['y'].cumsum()
    nCumsum = np.arange(len(y)) - pCumsum + 1
    pCumsumPer = pCumsum / PosAll
    nCumsumPer = nCumsum / NegAll
    TR1 = pCumsumPer[abs(nCumsumPer-0.001).idxmin()]
    TR2 = pCumsumPer[abs(nCumsumPer-0.005).idxmin()]
    TR3 = pCumsumPer[abs(nCumsumPer-0.01).idxmin()]
    return 0.4 * TR1 + 0.3 * TR2 + 0.3 * TR3

In [69]:
predictions1=lgbm.predict_proba(X_test[df_fi_col])[:,1]
tpr_weight_funtion1(y_test,predictions1)

0.7252913752913753

In [ ]:
# X_train1,X_test1,y_train1,y_test1=train_test_split(data_train9.iloc[:,1:-1],data_train9.iloc[:,-1],test_size=0.5,random_state=13)

lgbm=lgb.LGBMClassifier(
    colsample_bytree= 0.9374501116995294, 
    learning_rate= 0.06948460185031771, 
    #max_depth= 6,
    num_leaves= 96, 
    subsample= 0.8678848250908754,
    is_unbalance=True,
    n_estimators=500).fit(X_train,y_train)

predictions=lgbm.predict(X_test)
print('准确率',accuracy_score(y_test,predictions))
print('混淆矩阵',confusion_matrix(y_test,predictions))
print('分类报告',classification_report(y_test,predictions))

In [32]:
from sklearn.metrics import roc_curve,auc


In [ ]:
fpr,tpr,thresholds=roc_curve(y_test,predictions)
roc_auc=auc(fpr,tpr)
print('AUC is :{:.4f}'.format(roc_auc))

In [57]:
# ################### for all Train data
# ###  data_train9.iloc[:,1:-1],data_train9.iloc[:,-1]
# X_data=data_train9.iloc[:,1:-1]
# y_data=data_train9.iloc[:,-1]



# #use tpe to find best param
# from hyperopt import  hp,fmin,space_eval,tpe
# from sklearn.model_selection import cross_val_score,StratifiedKFold

# obj_call_count = 0
# def objective(params):
#     global obj_call_count
#     obj_call_count += 1
#     params={
#           'num_leaves':int(params['num_leaves']),
#           'min_data_in_leaf':int(params['min_data_in_leaf']),
#          # 'max_depth':int(params['max_depth']),
#           'colsample_bytree':'{:.4f}'.format(params['colsample_bytree']),
#           'learning_rate':'{:.4f}'.format(params['learning_rate']),
#           'subsample':'{:.4f}'.format(params['subsample']),
#           'min_sum_hessian_in_leaf':'{:.4f}'.format(params['min_sum_hessian_in_leaf']),
#           'max_bin':int(params['max_bin']),
#           'bagging_freq':int(params['bagging_freq']),
#           'lambda_l1':'{:.4f}'.format(params['lambda_l1']),
#           'lambda_l2':'{:.4f}'.format(params['lambda_l2']),
#           'n_estimators' :int(params['n_estimators'])
        
#     }
    
#     clf=lgb.LGBMClassifier(
#         #n_estimators=500,
#         is_unbalance=True,
#         n_jobs=3,
#         **params
#     )
    
#     score=-cross_val_score(clf,X_data,y_data,scoring=tpr_weight_scorer,cv=StratifiedKFold()).mean()
#     print('---------------{}-----------------------'.format(obj_call_count))
#     print('----tpe wieght: {:.3f}'.format(score))
#     #print(params)
#     print('-----------------------------------------')
#     return score


# space={
#     'num_leaves':hp.quniform('num_leaves',8,256,4),
#     'min_data_in_leaf':  hp.quniform ('min_data_in_leaf', 10, 100, 10),
#    # 'max_depth':hp.quniform('max_depth',2,30,1),
#     'colsample_bytree':hp.uniform('colsample_bytree',0.3,1),
#     'learning_rate':hp.uniform('learning_rate',0.001,0.5),
#     'subsample':hp.uniform('subsample',0.3,1),
#     'min_sum_hessian_in_leaf': hp.loguniform('min_sum_hessian_in_leaf', 0, 2.3),
#     'max_bin': hp.quniform ('max_bin', 128, 2048, 1),
#    # 'bagging_freq': hp.quniform ('bagging_freq', 1, 10, 1),
#     'lambda_l1': hp.uniform('lambda_l1', 0, 10 ),
#     'lambda_l2': hp.uniform('lambda_l2', 0, 10 ),
#     'n_estimators':hp.quniform('n_estimators',100,1000,100),
# }

# best=fmin(fn=objective,
#          space=space,
#          algo=tpe.suggest,
#          max_evals=500)

    

---------------1-----------------------
----tpe wieght: -0.592
-----------------------------------------
---------------2-----------------------
----tpe wieght: -0.589
-----------------------------------------
---------------3-----------------------
----tpe wieght: -0.503
-----------------------------------------
---------------4-----------------------
----tpe wieght: -0.569
-----------------------------------------
---------------5-----------------------
----tpe wieght: -0.581
-----------------------------------------
---------------6-----------------------
----tpe wieght: -0.558
-----------------------------------------
---------------7-----------------------
----tpe wieght: -0.513
-----------------------------------------
---------------8-----------------------
----tpe wieght: -0.590
-----------------------------------------
---------------9-----------------------
----tpe wieght: -0.558
-----------------------------------------
---------------10-----------------------
----tpe wieght

---------------79-----------------------
----tpe wieght: -0.580
-----------------------------------------
---------------80-----------------------
----tpe wieght: -0.589
-----------------------------------------
---------------81-----------------------
----tpe wieght: -0.523
-----------------------------------------
---------------82-----------------------
----tpe wieght: -0.577
-----------------------------------------
---------------83-----------------------
----tpe wieght: -0.583
-----------------------------------------
---------------84-----------------------
----tpe wieght: -0.587
-----------------------------------------


KeyboardInterrupt: 

In [89]:
print('Hyperopt estimated optimum {}'.format(best))

Hyperopt estimated optimum {'colsample_bytree': 0.7002886240992474, 'learning_rate': 0.09424641959240096, 'max_depth': 8.0, 'num_leaves': 58.0, 'subsample': 0.5281211159912743}


In [91]:

lgbm=lgb.LGBMClassifier(
    colsample_bytree= 0.7002886240992474, 
    learning_rate= 0.09424641959240096, 
    max_depth= 8,
    num_leaves= 58, 
    subsample= 0.5281211159912743,
    is_unbalance=True,
    n_estimators=500).fit(X_train,y_train)

predictions=lgbm.predict(X_test)
print('准确率',accuracy_score(y_test,predictions))
print('混淆矩阵',confusion_matrix(y_test,predictions))
print('分类报告',classification_report(y_test,predictions))

准确率 0.9810055865921787
混淆矩阵 [[2375    9]
 [  42  259]]
分类报告              precision    recall  f1-score   support

        0.0       0.98      1.00      0.99      2384
        1.0       0.97      0.86      0.91       301

avg / total       0.98      0.98      0.98      2685



D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [96]:
df_fi=pd.DataFrame()
df_fi['name']=X_train.columns.values
df_fi['importance']=lgbm.feature_importances_
df_fi.sort_values(by=['importance'],axis=0,ascending=False)


,name,importance
210,tr_trans_amt_max,870
82,mode_0,759
209,tr_trans_amt_min,642
211,tr_trans_amt_mean,579
278,df_op_last_day_se2_1.0_min,529
212,tr_trans_amt_median,486
0,op_count,480
1,tr_count,477
87,mode_5,473
281,df_op_last_day_se2_1.0_max,453


In [99]:
df_fi[df_fi['importance']>0].shape

(215, 2)

In [106]:
df_fi_col=df_fi[df_fi['importance']>0]['name'].values

In [107]:
df_fi_col

array(['op_count', 'tr_count', 'op_day_1', 'op_day_2', 'op_day_3',
       'op_day_4', 'op_day_5', 'op_day_6', 'op_day_7', 'op_day_8',
       'op_day_9', 'op_day_10', 'op_day_11', 'op_day_12', 'op_day_13',
       'op_day_14', 'op_day_15', 'op_day_16', 'op_day_17', 'op_day_18',
       'op_day_19', 'op_day_20', 'op_day_21', 'op_day_22', 'op_day_23',
       'op_day_24', 'op_day_25', 'op_day_26', 'op_day_27', 'op_day_28',
       'op_day_29', 'op_day_30', 'is_android', 'is_ios', 'success_0',
       'success_1', 'os_0', 'os_1', 'os_2', 'os_3', 'version_0',
       'version_1', 'version_2', 'version_3', 'version_4', 'version_5',
       'version_6', 'version_7', 'version_8', 'version_13', 'version_19',
       'mode_0', 'mode_1', 'mode_2', 'mode_3', 'mode_4', 'mode_5',
       'mode_6', 'mode_7', 'mode_8', 'mode_9', 'mode_10', 'mode_11',
       'mode_12', 'mode_13', 'mode_14', 'mode_15', 'mode_16', 'mode_17',
       'mode_18', 'mode_19', 'mode_20', 'mode_21', 'mode_22', 'mode_23',
       'mode_24'

In [108]:
X_data2=X_data[df_fi_col]
X_data2.head()

,op_count,tr_count,op_day_1,op_day_2,op_day_3,op_day_4,op_day_5,op_day_6,op_day_7,op_day_8,op_day_9,op_day_10,op_day_11,op_day_12,op_day_13,...,df_op_last2_se2_0.0_min,df_op_last2_se2_1.0_min,df_op_last2_se2_0.0_max,df_op_last2_se2_1.0_max,df_op_last2_se2_0.0_mean,df_op_last2_se2_1.0_mean,df_op_last2_se2_0.0_median,df_op_last2_se2_1.0_median,df_op_last3_se2_0.0_min,df_op_last3_se2_1.0_min,df_op_last3_se2_0.0_max,df_op_last3_se2_1.0_max,df_op_last3_se2_0.0_mean,df_op_last3_se2_1.0_mean,df_op_last3_se2_1.0_median
0,5,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,-999.0,13.0,-999.0,13.0,-999.0,13.0,-999.0,13.0,-999.0,79249.0,-999.0,79249.0,-999.0,79249.0,79249.0
1,65,18,0.0,11.0,14.0,5.0,4.0,0.0,8.0,0.0,0.0,0.0,12.0,4.0,7.0,...,-999.0,12.0,-999.0,12.0,-999.0,12.0,-999.0,12.0,-999.0,1.0,-999.0,1.0,-999.0,1.0,1.0
2,11,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-999.0,47.0,-999.0,47.0,-999.0,47.0,-999.0,47.0,-999.0,0.0,-999.0,0.0,-999.0,0.0,0.0
3,14,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-999.0,55.0,-999.0,55.0,-999.0,55.0,-999.0,55.0,-999.0,24.0,-999.0,24.0,-999.0,24.0,24.0
4,29,7,3.0,0.0,3.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,...,-999.0,69919.0,-999.0,69919.0,-999.0,69919.0,-999.0,69919.0,-999.0,3.0,-999.0,3.0,-999.0,3.0,3.0


In [112]:

lgbm=lgb.LGBMClassifier(
    colsample_bytree= 0.7002886240992474, 
    learning_rate= 0.09424641959240096, 
    max_depth= 8,
    num_leaves= 58, 
    subsample= 0.5281211159912743,
    is_unbalance=True,
    n_estimators=500).fit(X_data2,y_data)

predictions=lgbm.predict(X_test[df_fi_col])
print('准确率',accuracy_score(y_test,predictions))
print('混淆矩阵',confusion_matrix(y_test,predictions))
print('分类报告',classification_report(y_test,predictions))

准确率 1.0
混淆矩阵 [[2384    0]
 [   0  301]]
分类报告              precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      2384
        1.0       1.00      1.00      1.00       301

avg / total       1.00      1.00      1.00      2685



D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [125]:
predictions22=lgbm.predict(X_test[df_fi_col])
predictions23=lgbm.predict_proba(X_test[df_fi_col])[:,1]
print('准确率',accuracy_score(y_test,predictions22))
print('混淆矩阵',confusion_matrix(y_test,predictions22))
print('分类报告',classification_report(y_test,predictions22))

准确率 1.0
混淆矩阵 [[2384    0]
 [   0  301]]
分类报告              precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      2384
        1.0       1.00      1.00      1.00       301

avg / total       1.00      1.00      1.00      2685



D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [126]:
predictions22[:5]

array([0., 0., 0., 0., 0.])

In [127]:
predictions23[:5]

array([2.92541781e-07, 8.69209913e-06, 8.66273943e-06, 5.83078369e-05,
       2.08030525e-08])

In [128]:
result=pd.DataFrame()
result['UID']=data_test9['UID']
result['Tag']=lgbm.predict_proba(data_test9[df_fi_col])[:,1]

In [129]:
result.head()

,UID,Tag
13422,30000,0.407627
13423,30001,0.000435
13424,30002,0.002356
13425,30003,0.000043
13426,30004,0.005172


In [130]:
import datetime
nowTime=datetime.datetime.now().strftime('%Y%m%d%H%M%S')
name='TC_1_'+str(nowTime)
result.to_csv("F:\\study\\ml\\DataSet\\TianChengJinRong\\SubMission\\"+name+'.csv',index=False,encoding='utf-8')
